In [1]:
import tensorflow as tf

In [2]:
model = tf.keras.models.load_model('backbone_head_5')

In [3]:
'''
import os
import numpy as np
from collections import defaultdict
from sklearn.metrics import accuracy_score

import cv2
import numpy as np
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.utils import img_to_array

def model_predict(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img,(224,224))

    img = preprocess_input(img)
    
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array,0)
    prob = model.predict(img_array)
    return prob



def simple_voting_inference(video_directory, threshold=0.5,video_number = 0):
    actor_probabilities = defaultdict(list)

    count = 0
    # Iterate over each frame file in the video directory
    for frame_file in os.listdir(video_directory):
        frame_path = os.path.join(video_directory, frame_file)
        # Extract frame number and actor number from the file name
        _, actor_number = frame_file.split('.')[0].split('_')
        print(f'Video: {video_number} File: {count} Actor: {actor_number}')
        count+=1
        # Get the model's probability that the frame is real
        prob_real = model_predict(frame_path)
        # Append the probability to the actor's list of probabilities
        actor_probabilities[actor_number].append(prob_real)

    # Average the probabilities for each actor and apply the threshold
    actor_classification = {actor: 'Fake' if np.mean(probs) <= threshold else 'Real'
                            for actor, probs in actor_probabilities.items()}

    # Apply voting scheme: if any actor is classified as fake, the video is fake
    video_classification = 'Fake' if 'Fake' in actor_classification.values() else 'Real'

    return video_classification,actor_probabilities,actor_classification

def calculate_accuracy(val_directory, threshold=0.5):
    true_labels = []
    pred_labels = []

    count = 1
    # Process fake videos
    fake_videos_path = os.path.join(val_directory, 'FAKE')
    for video_name in os.listdir(fake_videos_path):
        video_path = os.path.join(fake_videos_path, video_name)
        video_classification,_,_ = simple_voting_inference(video_path, threshold,count)
        print(f'Fake Video {video_name} with count {count} classified as {video_classification}')
        count+=1
        true_labels.append('Fake')
        pred_labels.append(video_classification)

    # Process real videos
    real_videos_path = os.path.join(val_directory, 'REAL')
    for video_name in os.listdir(real_videos_path):
        video_path = os.path.join(real_videos_path, video_name)
        video_classification,_,_ = simple_voting_inference(video_path, threshold,count)
        print(f'Real Video {video_name} with count {count} classified as {video_classification}')
        count+=1
        true_labels.append('Real')
        pred_labels.append(video_classification)

    # Calculate accuracy
    label_encoder = {'Fake': 0, 'Real': 1}
    true_labels_encoded = [label_encoder[label] for label in true_labels]
    pred_labels_encoded = [label_encoder[label] for label in pred_labels]

    # Calculate accuracy
    accuracy = accuracy_score(true_labels_encoded, pred_labels_encoded)
    return accuracy

video_path = '../biometric_project/data/faces_subset_train_sample_videos_paper_implementation - Copy/val/REAL/aytzyidmgs/'
video_classification,actor_probabilities,actor_classification = simple_voting_inference(video_path)
print(video_classification)
print(actor_probabilities)
print(actor_classification)

accuracy_voting = calculate_accuracy('../biometric_project/data/faces_subset_train_sample_videos_paper_implementation - Copy/train/')
print(f"Accuracy: {accuracy_voting:.2f}")
'''


'\nimport os\nimport numpy as np\nfrom collections import defaultdict\nfrom sklearn.metrics import accuracy_score\n\nimport cv2\nimport numpy as np\nfrom tensorflow.keras.applications.efficientnet import preprocess_input\nfrom tensorflow.keras.utils import img_to_array\n\ndef model_predict(image_path):\n    img = cv2.imread(image_path)\n    img = cv2.resize(img,(224,224))\n\n    img = preprocess_input(img)\n    \n    img_array = img_to_array(img)\n    img_array = np.expand_dims(img_array,0)\n    prob = model.predict(img_array)\n    return prob\n\n\n\ndef simple_voting_inference(video_directory, threshold=0.5,video_number = 0):\n    actor_probabilities = defaultdict(list)\n\n    count = 0\n    # Iterate over each frame file in the video directory\n    for frame_file in os.listdir(video_directory):\n        frame_path = os.path.join(video_directory, frame_file)\n        # Extract frame number and actor number from the file name\n        _, actor_number = frame_file.split(\'.\')[0].split(

In [4]:
import os
import numpy as np
import cv2
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.utils import img_to_array
from sklearn.metrics import accuracy_score
import h5py

def model_predict(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = preprocess_input(img)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, 0)
    prob = model.predict(img_array)
    return prob

def standard_inference(video_path, model_predict, clas, threshold=0.5, video_number=0, h5_file=None):
    frame_predictions = []
    count = 0
    for frame_file in os.listdir(video_path):
        prob_real = model_predict(os.path.join(video_path, frame_file))
        print(f'Video: {video_number} File: {count}')
        count += 1
        frame_predictions.append(prob_real)
        
    if h5_file is not None:
        # Save the frame's probability to the HDF5 file
        h5_file.create_dataset(f'{clas}_video_{video_number}', data=frame_predictions)

    avg_prob_real = np.mean(frame_predictions)
    return 'Real' if avg_prob_real >= threshold else 'Fake'

def calculate_accuracy(val_directory, threshold=0.5, save_predictions=True, h5_filename=None):
    true_labels = []
    pred_labels = []

    with h5py.File(h5_filename, 'w') if save_predictions else dummy_context() as h5_file:
        count = 1
        fake_videos_path = os.path.join(val_directory, 'FAKE')
        for video_name in os.listdir(fake_videos_path):
            video_path = os.path.join(fake_videos_path, video_name)
            video_classification = standard_inference(video_path, model_predict,'Fake', threshold, count, h5_file)
            print(f'Fake Video {video_name} with count {count} classified as {video_classification}')
            count += 1
            true_labels.append('Fake')
            pred_labels.append(video_classification)

        real_videos_path = os.path.join(val_directory, 'REAL')
        for video_name in os.listdir(real_videos_path):
            video_path = os.path.join(real_videos_path, video_name)
            video_classification = standard_inference(video_path, model_predict,'Real', threshold, count, h5_file)
            print(f'Real Video {video_name} with count {count} classified as {video_classification}')
            count += 1
            true_labels.append('Real')
            pred_labels.append(video_classification)

    label_encoder = {'Fake': 0, 'Real': 1}
    true_labels_encoded = [label_encoder[label] for label in true_labels]
    pred_labels_encoded = [label_encoder[label] for label in pred_labels]

    accuracy = accuracy_score(true_labels_encoded, pred_labels_encoded)
    return accuracy

# Dummy context manager for optional HDF5 file handling
import contextlib
@contextlib.contextmanager
def dummy_context():
    yield None

# Example usage
accuracy = calculate_accuracy('../biometric_project/data/faces_test_videos/', save_predictions=True, h5_filename='predictions_test.h5')
print(f"Accuracy: {accuracy:.2f}")


1/1 [==============================] - 3s 3s/step
Video: 1 File: 0
1/1 [==============================] - 0s 63ms/step
Video: 1 File: 1
1/1 [==============================] - 0s 55ms/step
Video: 1 File: 2
1/1 [==============================] - 0s 47ms/step
Video: 1 File: 3
1/1 [==============================] - 0s 62ms/step
Video: 1 File: 4
1/1 [==============================] - 0s 47ms/step
Video: 1 File: 5
1/1 [==============================] - 0s 47ms/step
Video: 1 File: 6
1/1 [==============================] - 0s 47ms/step
Video: 1 File: 7
1/1 [==============================] - 0s 47ms/step
Video: 1 File: 8
1/1 [==============================] - 0s 47ms/step
Video: 1 File: 9
1/1 [==============================] - 0s 51ms/step
Video: 1 File: 10
1/1 [==============================] - 0s 57ms/step
Video: 1 File: 11
1/1 [==============================] - 0s 47ms/step
Video: 1 File: 12
1/1 [==============================] - 0s 47ms/step
Video: 1 File: 13
1/1 [===========================

In [5]:
import h5py
import numpy as np

# Path to your HDF5 file
h5_filename = 'predictions_test.h5'

# Open the HDF5 file in read mode
with h5py.File(h5_filename, 'r') as h5_file:
    # Iterate over each dataset in the file
    for video_number in h5_file.keys():
        # Retrieve the array of probabilities for the current video
        frame_probabilities = np.array(h5_file[video_number])
        print(f"{video_number}: {frame_probabilities}")


Fake_video_1: [[[2.38898374e-05]]

 [[4.60540317e-02]]

 [[1.03575451e-06]]

 [[1.56993382e-02]]

 [[1.88938465e-08]]

 [[2.48760339e-02]]

 [[1.05679099e-07]]

 [[1.67757981e-02]]

 [[4.81230558e-08]]

 [[2.12033987e-02]]

 [[7.34469410e-08]]

 [[1.71723962e-02]]

 [[1.80578047e-06]]

 [[1.72992080e-01]]

 [[1.97099234e-05]]

 [[1.24764867e-01]]

 [[2.02633219e-05]]

 [[1.18015103e-01]]

 [[8.11013308e-07]]

 [[1.47698060e-01]]

 [[4.06193158e-06]]

 [[5.58893662e-03]]

 [[1.45884860e-05]]

 [[4.14743461e-02]]

 [[1.82449980e-06]]

 [[3.45406309e-02]]

 [[1.22247984e-05]]

 [[3.39152552e-02]]

 [[8.99446150e-06]]

 [[5.38841300e-02]]

 [[1.91251888e-06]]

 [[6.50215447e-02]]

 [[5.30776833e-06]]

 [[9.56861377e-02]]

 [[7.83792809e-02]]

 [[1.02581573e-04]]

 [[1.47502226e-06]]

 [[5.63888699e-02]]

 [[1.57231523e-06]]

 [[4.26385291e-02]]

 [[1.88553270e-06]]

 [[2.06667677e-01]]

 [[9.87310523e-06]]

 [[1.72744006e-01]]

 [[1.65657948e-05]]

 [[1.24033540e-02]]

 [[5.82460889e-05]]


In [6]:
import h5py
import numpy as np

# Path to your HDF5 file
h5_filename = 'predictions_test.h5'

# Dictionary to hold all videos' probabilities, with varying lengths
video_probabilities = {}

# Open the HDF5 file in read mode
with h5py.File(h5_filename, 'r') as h5_file:
    # Use enumerate to get both the index and the dataset key
    for index, key in enumerate(h5_file.keys()):
        # Retrieve and flatten the array of probabilities for the current video
        frame_probabilities = np.array(h5_file[key]).flatten()
        
        # Use the index as the key in the dictionary
        video_probabilities[index] = frame_probabilities

# Print the probabilities for each video
for index, probabilities in video_probabilities.items():
    print(f"Video {index}: {probabilities}")


Video 0: [2.38898374e-05 4.60540317e-02 1.03575451e-06 1.56993382e-02
 1.88938465e-08 2.48760339e-02 1.05679099e-07 1.67757981e-02
 4.81230558e-08 2.12033987e-02 7.34469410e-08 1.71723962e-02
 1.80578047e-06 1.72992080e-01 1.97099234e-05 1.24764867e-01
 2.02633219e-05 1.18015103e-01 8.11013308e-07 1.47698060e-01
 4.06193158e-06 5.58893662e-03 1.45884860e-05 4.14743461e-02
 1.82449980e-06 3.45406309e-02 1.22247984e-05 3.39152552e-02
 8.99446150e-06 5.38841300e-02 1.91251888e-06 6.50215447e-02
 5.30776833e-06 9.56861377e-02 7.83792809e-02 1.02581573e-04
 1.47502226e-06 5.63888699e-02 1.57231523e-06 4.26385291e-02
 1.88553270e-06 2.06667677e-01 9.87310523e-06 1.72744006e-01
 1.65657948e-05 1.24033540e-02 5.82460889e-05 2.71240145e-01
 3.36901030e-06 4.98256177e-01 6.56308066e-06 7.73304850e-02
 3.43036663e-06 2.07384735e-01 3.08831909e-06 3.76767576e-01
 2.57988631e-06 4.05522287e-02 1.10932624e-05 8.26437250e-02
 4.29867506e-02 2.45528208e-05 6.20253513e-06 3.57804410e-02
 9.66307052e-05

In [7]:
print(video_probabilities[10].shape)

(299,)
